In [ ]:
#@title
#@markdown <h3>⬅️ Click this First</h3>
import IPython
from google.colab import output
 
display(IPython.display.Javascript('''
function ClickConnect(){
  btn = document.querySelector("colab-connect-button")
  if (btn != null){
    console.log("Click colab-connect-button"); 
    btn.click() 
    }
    
  btn = document.getElementById('ok')
  if (btn != null){
    console.log("Click reconnect"); 
    btn.click() 
    }
  }
    
setInterval(ClickConnect,60000)
'''))

<IPython.core.display.Javascript object>

# <center><b> Put Your Cloud Drives </center>

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

<center>Rclone Mount/Unmount/Copy </center>
<center><img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>

In [ ]:
#@title <<----<strong>Install rClone</strong>
%%capture
AUTO_RECONNECT = False #@param {type:"boolean"}
RCLONE = True #@param {type:"boolean"}
#@markdown Check AUTO_RECONNECT to prevent notebook from disconnecting!
 
from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True) 
  
if RCLONE==True:
  !curl https://rclone.org/install.sh | sudo bash
 
if AUTO_RECONNECT:
  import IPython
  from google.colab import output
 
  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button"); 
      btn.click() 
      }
    
    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect"); 
      btn.click() 
      }
    }
    
  setInterval(ClickConnect,60000)
  '''))

In [ ]:
#@title <-----<strong>Upload your  Rclone.config file</strong>
def moveConfig():
  !mv rclone.conf /root/.config/rclone/rclone.conf
 
from google.colab import files
 
uploaded = files.upload()
 
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
moveConfig()
print("Moved rclone.conf to /root/.config/rclone/rclone.conf")

Saving rclone.conf to rclone.conf
User uploaded file "rclone.conf" with length 6235 bytes
Moved rclone.conf to /root/.config/rclone/rclone.conf


In [ ]:
#@markdown <center><h3>Rclone MOUNT / UNMOUNT</h3>Mount the remote as file system on a mountpoint.</center>
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
##########################################
 
class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Rclone available config...</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))
content = open("/root/.config/rclone/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)
cache_path="/content/temp/rCloneTemp"
def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    !rclone mount $mountNam.value: $mPoint --user-agent 'Mozilla' --buffer-size 256M --transfers 10 --vfs-cache-mode minimal --vfs-read-chunk-size 500M --vfs-cache-max-size 50G --vfs-cache-max-age 0h0m1s --vfs-cache-poll-interval 0m1s --cache-dir '/content/temp/rCloneTemp' --allow-other --daemon 
 
    if os.path.isdir(mPoint)== True:
      print(f"Mount success! - \t{mPoint}")
    else:
      print(f"Mount failed! - \t{mPoint}")
 
def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Unmounted success! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)
 
ShowAC()

Mount success! - 	/content/drives/Dropbox


<b>==================================================================</b>

# <center><b> Transfer Files & Folders </center>

<b><I>*** Copy Files

In [ ]:
#@markdown *Input file will be copied into the destination folder.*
input_file = "" #@param {type:"string"}
destination_folder = "" #@param {type:"string"}
!cp -v "$input_file" "$destination_folder"
print("Copied successfully!")

<b><I>*** Copy Folders

In [ ]:
#@markdown *Input folder will be copied into the destination folder.*
input_folder = "" #@param {type:"string"}
destination_folder = "" #@param {type:"string"}
!cp -a -v "$input_folder" "$destination_folder"
print("Copied successfully!")

# <center><b> Torrent Only </center>

In [ ]:
#Run This First
!pip install qiskit ipywidgets
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!pip install folium==0.2.1
 
#@title
#@markdown <h3>⬅️ Click Here to install libtorrent</h3>
 
 
 
!apt install python3-libtorrent
 
import libtorrent as lt
 
ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []
 
from IPython.display import HTML, clear_output
 
clear_output()
 
print("Server Started Successfully")

In [ ]:
#@title
#@markdown <h3>Enter files torrent and the path :</h3>
path = "/content/drive/Shared drives/sidc" #@param {type:"string"}
check=[]
 
 
from google.colab import files
 
source = files.upload()
params = {
    "save_path": path,
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))
 
clear_output()
 
print("Started Successfully")

In [ ]:
#@title
#@markdown <h3>Enter the path and Add magnet link :</h3>
path = "/content/drive/Shared drives/Google_Drive/Torrent" #@param {type:"string"}
check=[]
params = {"save_path": path}
 
while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )
 
clear_output()
 
print("Started Successfully")

In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to Start Download</h3>
import time
from IPython.display import display
import ipywidgets as widgets
 
state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]
 
layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)
 
while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()
 
            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)

#<center><b> Compress/Decompress Files in Cloud Drives </center>

##__Unzip/Unrar/Untar Files__

###__Unzip Files__

*   First path in below cell - Your Google Drive file location which you want to Unzip
*  Second path in below cell - Path where you want to extract the zip file in Google Drive

In [ ]:
!unzip "/content/drive/path/input_file_name.zip" -d "/content/drive/path/output_folder/"

###__Unrar Files</center>__

*   First path in below cell - Your Google Drive file location which you want to Unrar
*   Second path in below cell - Path where you want to extract the rar file in Google Drive
 
*If you want to extract a password protected rar file, then add "-Y" flag or Use this command instead of the command in the cell
(replace the command with the command in the cell)*
 
* ```
!unrar x -Y "/content/drive/path/input_file_name.rar" "/content/drive/path/output_folder/"
```

In [ ]:
!unrar x "/content/drive/path/input_file_name.rar" "/content/drive/path/output_folder/"

###__Untar Files__

*   First path in below cell - Your Google Drive file location which you want to Untar
*   Second path in below cell - Path where you want to extract the tar file in Google Drive
 
 *__Flags__*
 
 * -x : Extract a tar ball.
 * -v : Verbose output or show progress while extracting files.
 * -f : Specify an archive or a tarball filename.
 * -C : Specify a different directory to extract
 * -z : Decompress and extract the contents of the compressed archive created by gzip program (tar.gz extension).
 * -j : Decompress and extract the contents of the compressed archive created by bzip2 program (tar.bz2 extension).

In [ ]:
!tar -xvf "/content/drive/path/input_file_name.tar" -C "/content/drive/path/output_folder/"     #[run this cell to extract tar files]

In [ ]:
!tar -xzvf "/content/drive/path/input_file_name.tar.gz" -C "/content/drive/path/output_folder/"     #[run this cell to extract tar.gz files]

In [ ]:
!tar -xjvf "/content/drive/path/input_file_name.tar.bz2" -C "/content/drive/path/output_folder/"     #[run this cell to extract tar.bz2 files]

##__Zip/Rar/Tar Files/Folders__

###__Zip Files/Folders__

*   First path in below cell - Your Zip output filename
*   Second path in below cell - Google Drive file location of the files which you want to Zip

In [ ]:
!zip "/content/drive/path/output_file_name.zip" "/content/drive/My Drive/Tutorial/games"

###__Rar Files/Folders__

In [ ]:
# <= Click this cell to rar the folder
 
!apt-get install rar

*   First path in below cell - Your Rar File output Name (do not put .rar at the end. It will take it automatically)
*   Second path in below cell - Google Drive file location of the files/folder which you want to Rar
 
*If you want to create a password protected rar file, then add "-p" flag or Use this command instead of the command in in the cell.
(replace the command with the command in the cell) After running the cell, it will ask you to provide the password*
 
* ```
!rar a -p "/content/drive/path/output_file_name" "/content/drive/path/input_file_or_folder"
```

In [ ]:
!rar a "/content/drive/path/output_file_name" "/content/drive/path/input_file_or_folder"

###__Tar Files/Folders__

*   First path in below cell - Your Tar output filename
*   Second path in below cell - Google Drive file location of the files which you want to Tar
 
 *__Flags__*
 
 * -c : Create a new archive
 * -v : Verbose output
 * -f : Use the given archive file (name.extention)
 * -z : Filter the archive through gzip
 * -j : Filter the archive through bzip2

In [ ]:
!tar -cvf "/content/drive/path/output_file_name.tar" "/content/drive/path/input_file_or_folder"     #[run this cell to create tar files]

In [ ]:
!tar -czvf "/content/drive/path/output_file_name.tar.gz" "/content/drive/path/input_file_or_folder"     #[run this cell to create tar.gz files]

In [ ]:
!tar -cjvf "/content/drive/path/output_file_name.tar.bz2" "/content/drive/path/input_file_or_folder"      #[run this cell to create tar.bz2 files]